In [2]:
# import des librairies
import pandas as pd 
import numpy as np
import math

# srapping
import requests 
from bs4 import BeautifulSoup as bs
import re

# clean data
import re

#export data
import time
import os
from datetime import date
from datetime import datetime

In [14]:
# fonctions
def scrapping():
    job_offers = []
    page = 0
    # Select URL
    url = f'https://candidat.pole-emploi.fr/offres/recherche?&motsCles=data&offresPartenaires=true&range=0-19&rayon=10&tri=0'
    while True:
        
        response = requests.get(url)
        
#         # Check if the URL is not recognized or does not exist
#         if response.status_code != 200:
#             break
            
        html_content = response.text
            
        with open ('page.html', 'w', encoding='utf-8') as file:
            file.write(html_content)

        with open('page.html', 'r', encoding='utf-8') as file:
            html_content = file.read()

        soup = bs(html_content, 'html.parser')

        job_elements = soup.find_all('div', class_='media-body')
        
        for job_element in job_elements:
            # Extract the data-intitule-offre value from the job summary element
            
            if job_element.find('h2') and job_element.find('h2')['data-intitule-offre']:
                data_intitule_offre = job_element.find('h2')['data-intitule-offre']
            
                # Construct the detail page URL using the data-intitule-offre value
                detail_page_url = f'https://candidat.pole-emploi.fr/offres/recherche/detail/{data_intitule_offre}'

                # Scrape job details from the detail page
                detail_page_response = requests.get(detail_page_url)
                detail_page_content = detail_page_response.text

                with open ('page.html', 'w', encoding='utf-8') as file:
                    file.write(detail_page_content)

                with open('page.html', 'r', encoding='utf-8') as file:
                    detail_page_content = file.read()

                detail_page_soup = bs(detail_page_content, 'html.parser')
                
                # Liste des variables à extraire
                variables = {
                    'jobs': ('span', 'title'),
                    'description': ('div', 'description col-sm-8 col-md-7'),
                    'loc': ('span', 'name'),
                    'datePosted': ('span', 'datePosted'),
                    'workhours': ('dd', 'workHours'),
                    'salary': ('span', 'baseSalary'),
                    'contract_type': ('dl', 'icon-group'),
                    'experience': ('span', 'experienceRequirements'),
                    'qualifications': ('span', 'qualifications'),
                    'industry': ('span', 'industry'),
                    'company': ('h3', 't4 title'),
                    'size': ('div', 'media-body'),
                    'company_description': ('p', 'italic'),
                    'diploma': ('span', 'educationRequirements')
                }

                # Boucle pour extraire les variables
                for var, (tag, class_) in variables.items():
                    try:
                        if var in ['salary', 'contract_type']:
                            if var == 'salary':
                                try:
                                    salary_raw = detail_page_soup.find(tag, itemprop=class_)
                                    min_value = salary_raw.find('span', itemprop='minValue').get('content').strip()
                                    max_value = salary_raw.find('span', itemprop='maxValue').get('content').strip()
                                    globals()[var] = f"{min_value} - {max_value}"
                                except AttributeError:
                                    salary_match = re.search(r'content="([\d.]+)" itemprop="value"', str(salary_raw))
                                    globals()[var] = salary_match.group(1) if salary_match else math.nan
                                    
                            elif var == 'contract_type':
                                contract_type_raw = detail_page_soup.find(tag, class_=class_).find('dd')
                                globals()[var] = str(contract_type_raw.contents[0]).strip()
                                
                        else:
                            globals()[var] = detail_page_soup.find(tag, itemprop=class_).text.replace('\n', '').strip()
                    except AttributeError:
                        # En cas d'erreur
                        globals()[var] = math.nan


                # Extraction des autres variables spécifiques
                try:
                    skills_raw = detail_page_soup.find_all('span', itemprop='skills')
                    skills = [skill.text.strip() for skill in skills_raw]
                except AttributeError:
                    # En cas d'erreur
                    skills_raw = ''
                    skills = math.nan
                    
                try:
                    company = detail_page_soup.find('h3', class_='t4 title').text.strip()
                except AttributeError:
                    company = math.nan
                    
                try:
                    size = detail_page_soup.find('div', class_='media-body').find('p').text.strip()
                except AttributeError:
                    size = math.nan
                    
                try:                    
                    company_description = detail_page_soup.find('p', class_='italic').text.replace('\n', '').strip()
                except AttributeError:
                    company_description = math.nan
                
                try: 
                    description = detail_page_soup.find('div', class_='description col-sm-8 col-md-7').text.strip()
                except AttributeError:
                    description = math.nan
                    

                # Create a dictionary for the scraped data
                data = {
                    'jobs': jobs,
                    'description': description,
                    'loc': loc,
                    'datePosted': datePosted,
                    'workhours': workhours,
                    'salary': salary,
                    'contract_type': contract_type,
                    'experience': experience,
                    'diploma': diploma,
                    'skills': skills,
                    'qualifications': qualifications,
                    'industry': industry,
                    'company': company,
                    'size': size,
                    'company_description': company_description
                }

                job_offers.append(data)

            else : 
                print(f'fin de la page {page}!')

        # Extract the href of the next page button
        next_page_link = soup.find('div', id='zoneAfficherPlus').find('a')
        if next_page_link:
            href_main_page = next_page_link['href']
        else:
            break
        
        # Construct the URL for the next page
        next_page_url = f'https://candidat.pole-emploi.fr{href_main_page}'
        
        # Delay before navigating to the next page
        time.sleep(60)
        
        # Update the URL to the next page
        url = next_page_url
        page += 1

    # Transform the dictionary into a dataframe
    df = pd.DataFrame(job_offers)
    
    return df

In [15]:
df = scrapping()

fin de la page 0!
fin de la page 1!
fin de la page 2!
fin de la page 3!
fin de la page 4!
fin de la page 5!
fin de la page 6!
fin de la page 7!
fin de la page 8!
fin de la page 9!
fin de la page 10!
fin de la page 11!
fin de la page 12!
fin de la page 13!
fin de la page 14!
fin de la page 15!
fin de la page 16!
fin de la page 17!
fin de la page 18!
fin de la page 19!
fin de la page 20!
fin de la page 21!
fin de la page 22!
fin de la page 23!
fin de la page 24!
fin de la page 25!
fin de la page 26!
fin de la page 27!
fin de la page 28!
fin de la page 29!
fin de la page 30!
fin de la page 31!
fin de la page 32!
fin de la page 33!
fin de la page 34!
fin de la page 35!
fin de la page 36!
fin de la page 37!
fin de la page 38!
fin de la page 39!
fin de la page 40!
fin de la page 41!
fin de la page 42!
fin de la page 43!
fin de la page 44!
fin de la page 45!
fin de la page 46!
fin de la page 47!
fin de la page 48!
fin de la page 49!


In [13]:
display(df.head(3))
print(df.shape)

,jobs,description,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description,tools,ID_dep,ville,date
0,ANALYSTE APPLICATIF DATA FINANCE RH,Dans le cadre d'un important projet avec un gr...,35H Travail en journée,50000.0 - 60000.0,Contrat à durée indéterminée,2 ans,NaN,Adapter les outils de traitement statistique d...,Cadre,Conseil en systèmes et logiciels informatiques,INFORMATIS T.S.,6 à 9 salariés,NaN,NaN,92,ISSY LES MOULINEAUX,2023/08/31
1,Alternant(e) en data analyse,La Caisse Primaire d'Assurance Maladie du Tarn...,35H Travail en journée,NaN,Contrat à durée déterminée - 12 Mois,Débutant accepté,"Bac+3, Bac+4 ou équivalents",Adapter les outils de traitement statistique d...,Employé qualifié,Activités générales de sécurité sociale,CPAM,200 à 249 salariés,La CPAM de Tarn et Garonne gère une population...,tableau,82,MONTAUBAN,2023/08/31
2,Alternant Data Analyst,******* Un poste d'Alternant Data Analyst est ...,35H Travail en journée,NaN,Contrat à durée déterminée - 24 Mois,6 mois,"Bac+3, Bac+4 ou équivalents - Ecole d'ingénieu...",Adapter les outils de traitement statistique d...,"Ouvrier qualifié (P1,P2)",Construction de navires et de structures flott...,CATANA GROUP,1 ou 2 salariés,L'entreprise :Le Groupe Catana est une société...,"tableau, power bi",17,MARANS,2023/08/31


(688, 17)


In [69]:
df.shape

(688, 17)

### Export df

In [7]:
# def export_raw_data(df):
#     csv_file_path = f"../data/bronze/raw_data.csv"
#     df.to_csv(csv_file_path, index=False)

In [8]:
# export_raw_data(df)

## Transform data

In [107]:
tools_list = ["AWS S3", "Azure Blob Storage", "Google Cloud Storage", "Dropbox", "Box", "OneDrive", "Salesforce", "HubSpot",
               "Slack", "Microsoft Teams", "Google Workspace", "Trello", "Jira", "Confluence", "GitHub", "GitLab", "Jenkins",
               "CircleCI", "Travis CI", "Docker", "Kubernetes", "Apache Hadoop", "Apache Spark", "Apache Kafka", "Apache Storm",
               "Elasticsearch", "Logstash", "Kibana", "Grafana", "Prometheus", "Nagios", "Splunk", "Tableau", "Power BI",
               "QlikView", "Looker", "Dataiku", "Alteryx", "Talend", "Informatica", "Matillion", "Databricks", "Snowflake",
               "Amazon Redshift", "Google BigQuery", "Microsoft Azure SQL Database", "Oracle Database", "MySQL", "PostgreSQL",
               "MongoDB", "Couchbase", "Cassandra", "Neo4j", "Apache Airflow", "Luigi", "Celery", "Apache NiFi", 
               "Talend Data Integration", "Google Cloud Dataflow", "Apache Beam", "Apache Flink", 
               "Microsoft Azure Data Factory", "IBM InfoSphere DataStage", "Apache Samza", "Apache Flume",
               "Apache Sqoop", "Microsoft Azure Databricks", "Teradata", "IBM Db2", "Apache Zeppelin", 
               "Jupyter Notebook", "Google Colab", "Anaconda", "TensorFlow", "PyTorch", "Keras",
               "scikit-learn", "XGBoost", "LightGBM", "H2O.ai", "Apache Mahout", "Microsoft Azure Machine Learning", 
               "IBM Watson", "RapidMiner", "KNIME", "DataRobot", "Amazon SageMaker", "Google Cloud AI Platform",
               "Domo", "IBM Cognos", "SAS", "Oracle Analytics Cloud", "Microsoft Power Automate",
               "Apache NiFi Registry", "Apache Atlas", "Apache Metron", "Apache Knox", "Apache Ranger",
               "Apache Superset", "Microsoft Power Apps", "Salesforce Einstein Analytics",
               "Google Data Studio", "Pentaho", "Microsoft Azure Synapse Analytics",
               "SAP Analytics Cloud", "MicroStrategy", "RStudio", "Apache Kylin", 
               "Apache HBase", "Apache Pig", "Apache Hive", "Apache Drill",
               "Cloudera Data Platform", "Hortonworks Data Platform", "Qubole", "DataRobot", "DataRobot Insights",
               "DataRobot MLOps", "DataRobot Paxata", "DataRobot AutoML", "DataRobot Time Series", "DataRobot AI Catalog"]

In [108]:
def create_tools(df, liste):
    tools_list = [tool.lower() for tool in liste]
    
    def find_tools(description):
        # Convertir la description en minuscules
        description = description.lower()

        # Créer une liste pour stocker les outils trouvés
        tools_found = []

        # Chercher chaque outil dans la description
        for tool in tools_list:
            if tool in description:
                tools_found.append(tool)
        
        # Si aucun outil n'a été trouvé, retourner NaN
        if not tools_found:
            return math.nan

        return ', '.join(tools_found)  # Retourner la liste d'outils trouvés
    
    df['tools'] = df['description'].apply(find_tools)
    return df

In [109]:
def new_skills(df):
    
    def clean_skills(skills_list):
        # Vérifier si la valeur est NaN
        if pd.isna(skills_list):
            return skills_list

        # Si la valeur est une chaîne, retournez-la telle quelle
        if isinstance(skills_list, str):
            return skills_list

        # Convertir chaque compétence en minuscules (si vous le souhaitez toujours)
        clean_list = [skill.lower() for skill in skills_list]
            
        # Convertir la liste en une seule chaîne de caractères
        return ', '.join(clean_list)
    
    df['skills'] = df['skills'].apply(clean_skills)
    
    return df

In [110]:
def new_skills_v2(df):
    
    def clean_skills(skills_list_or_str):
        # Vérifier si la valeur est NaN
        if pd.isna(skills_list_or_str):
            return skills_list_or_str

        # Si la valeur est une chaîne représentant une liste, convertissez-la en liste
        if isinstance(skills_list_or_str, str) and skills_list_or_str.startswith('[') and skills_list_or_str.endswith(']'):
            try:
                skills_list = ast.literal_eval(skills_list_or_str)
            except (ValueError, SyntaxError):
                return skills_list_or_str
        else:
            skills_list = skills_list_or_str

        # Convertir chaque compétence en minuscules (si vous le souhaitez toujours)
        clean_list = [skill.lower() for skill in skills_list]
            
        # Convertir la liste en une seule chaîne de caractères
        return ', '.join(clean_list)
    
    df['skills'] = df['skills'].apply(clean_skills)
    
    return df

In [111]:
df = create_tools(df, tools_list)

In [112]:
# df = new_skills(df)

In [113]:
df.head()

,jobs,description,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description,tools,ID_dep,ville,date
0,ANALYSTE APPLICATIF DATA FINANCE RH,Dans le cadre d'un important projet avec un gr...,35H Travail en journée,50000.0 - 60000.0,Contrat à durée indéterminée,2 ans,NaN,Adapter les outils de traitement statistique d...,Cadre,Conseil en systèmes et logiciels informatiques,INFORMATIS T.S.,6 à 9 salariés,NaN,NaN,92,ISSY LES MOULINEAUX,2023/08/31
1,Alternant(e) en data analyse,La Caisse Primaire d'Assurance Maladie du Tarn...,35H Travail en journée,NaN,Contrat à durée déterminée - 12 Mois,Débutant accepté,"Bac+3, Bac+4 ou équivalents",Adapter les outils de traitement statistique d...,Employé qualifié,Activités générales de sécurité sociale,CPAM,200 à 249 salariés,La CPAM de Tarn et Garonne gère une population...,tableau,82,MONTAUBAN,2023/08/31
2,Alternant Data Analyst,******* Un poste d'Alternant Data Analyst est ...,35H Travail en journée,NaN,Contrat à durée déterminée - 24 Mois,6 mois,"Bac+3, Bac+4 ou équivalents - Ecole d'ingénieu...",Adapter les outils de traitement statistique d...,"Ouvrier qualifié (P1,P2)",Construction de navires et de structures flott...,CATANA GROUP,1 ou 2 salariés,L'entreprise :Le Groupe Catana est une société...,"tableau, power bi",17,MARANS,2023/08/31
3,Chef(fe) de projets data management,Vous aurez pour mission d'assurer le suivi des...,35H Travail en journée,30000.0 - 36000.0,Contrat à durée indéterminée,1 an,Bac+2 ou équivalents,"Concevoir et gérer un projet,Concevoir un logi...",Cadre,Recherche-développement en sciences humaines e...,EUROFINS OPTIMED,50 à 99 salariés,"EUROFINS OPTIMED, centre de recherche Clinique...",NaN,38,GIERES,2023/08/31
4,Collaborateur comptable / DATA contrôleur,Dans un environnement en pleine transformation...,35H Travail en journée,NaN,Contrat à durée indéterminée,1 an,NaN,"Audit comptable,Audit comptable et financier,C...",Agent de maîtrise,Activités des sièges sociaux,LIDL,250 à 499 salariés,NaN,NaN,67,STRASBOURG,2023/08/31


In [115]:
def transform_data(df):
    df = df.copy()
    
    df[['ID_dep', 'ville']] = df['loc'].str.split(' - ', expand = True)
    df['date'] = df['datePosted'].str.replace('Actualisé le ', '').str.replace('Publié le ', '')
    df['date'] = df['date'].apply(lambda x: dateparser.parse(x, languages=['fr']))
    df['date'] = df['date'].dt.strftime('%Y/%m/%d')
    
    df["jobs"] = df["jobs"].str.replace(" h/f", "")
    df["jobs"] = df["jobs"].str.replace("\s*\([hH]/[fF]\)\s*", " ").str.strip()
    df["jobs"] = df["jobs"].str.replace(" (H/F)", "")
    df["jobs"] = df["jobs"].str.replace(" H/F", "")
    df["jobs"] = df["jobs"].str.replace(" F/H", "")
    df["jobs"] = df["jobs"].str.replace(" (F/H)", "")
    df["jobs"] = df["jobs"].str.replace(" \(\)", "")

    
    df = df.applymap(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)
    df = df.drop(["loc", "datePosted"], axis=1)

    return df

In [79]:
# df = transform_data(df)

In [116]:
df.head()
print(df.shape)

(688, 17)


In [117]:
def count_nan(df):
    
    nan_counts = df.isna().sum() # compte le nombre de NaN pour chaque colonne
    total_counts = len(df) # compte le nombre total de données dans le dataframe
    nan_percentages = (nan_counts / total_counts) * 100 # calcule le pourcentage de NaN pour chaque colonne
    result_df = pd.concat([nan_counts, nan_percentages], axis=1) # combine les deux séries en un dataframe
    result_df.columns = ['NaN Count', 'NaN Percentage'] # renomme les colonnes du nouveau dataframe
    result_df = result_df.sort_values(by = ['NaN Count'], ascending = False)
    return result_df

In [118]:
count_nan(df)

,NaN Count,NaN Percentage
diploma,660,95.930233
skills,635,92.296512
workhours,607,88.226744
salary,589,85.610465
tools,320,46.511628
qualifications,297,43.168605
company,286,41.569767
company_description,273,39.680233
industry,261,37.936047
size,203,29.505814


### Export Transform df 

In [121]:
def export_silver_to_csv(df):
    # Obtenez la date actuelle
    current_date = date.today().strftime("%d_%m_%Y")

    # Assurez-vous que le dossier de destination existe
    folder_path = f"../data/silver/{current_date}"
    os.makedirs(folder_path, exist_ok=True)

    # Exportez le DataFrame dans un fichier CSV
    csv_file_path = f"{folder_path}/jobs_data.csv"
    df.to_csv(csv_file_path, index=False)

In [122]:
export_silver_to_csv(df)

### Concat silver and transform df

In [124]:
jobs_df = pd.read_csv("../data/silver_data.csv")
display(jobs_df.tail(3))
print(jobs_df.shape)

,jobs,description,loc,datePosted,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description,tools,ID_dep,ville,date
3296,Développeur / Développeuse Big Data,Nous recherchons pour l'un de nos clients un(e...,NaN,NaN,NaN,NaN,Profession libérale,Expérience exigée de 5 An(s),NaN,NaN,NaN,NaN,ALLEGIS GROUP,NaN,NaN,"gitlab, docker, kubernetes, grafana, prometheu...",75,PARIS 01,2023/08/31
3297,Développeur / Développeuse Big Data,Contexte / Objectifs : \nAu sein de la platefo...,NaN,NaN,NaN,NaN,Profession libérale,Expérience exigée de 5 An(s),NaN,NaN,NaN,NaN,HN SERVICES,NaN,NaN,"power bi, informatica",75,PARIS 01,2023/08/31
3298,Développeur / Développeuse Big Data,Ton quotidien chez Komeet sera rythmé par :\n\...,NaN,NaN,NaN,36000.0 - 75000.0,Contrat à durée indéterminée,Expérience exigée,NaN,NaN,NaN,NaN,KOMEET TECHNOLOGIES,NaN,NaN,NaN,75,PARIS 01,2023/08/31


(3299, 19)


In [84]:
jobs_df["skills"][1]

"analyser les résultats d'un projet, décliner la conception générale d'un projet architectural en plans et maquettes, déterminer les modalités de chantier, esquisse d'un projet d'édifice ou d'espace, réaliser l'esquisse d'un projet de création ou d'aménagement, réceptionner un ouvrage"

In [85]:
jobs_df.head()

,jobs,description,loc,datePosted,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description,tools,ID_dep,ville,date
0,Data Engineer sénior (F/H) CDI (H/F),"En tant que Data Engineer chez Quantmetry, vou...",75 - PARIS 08,Actualisé le 12 juillet 2023,38H Travail en journée,40000.0 - 50000.0,Contrat à durée indéterminée,5 ans,NaN,"concevoir et gérer un projet, concevoir un log...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
1,Data Consultant Stratégie Sénior (H/F),Nous recrutons des personnes avec une appétenc...,75 - PARIS 08,Actualisé le 12 juillet 2023,35H Travail en journée,3100.0 - 5000.0,Contrat à durée indéterminée,5 ans,NaN,"analyser les résultats d'un projet, décliner l...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
2,Chef de projets Performance Durable/Energie/Da...,Intégré(e) au sein de la Direction Performance...,92 - ASNIERES SUR SEINE,Actualisé le 13 juillet 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,3 ans,NaN,contrôler et faire appliquer le respect de dis...,Employé qualifié,Activités des sièges sociaux,NEXITY,250 à 499 salariés,Nexity est aujourd hui leader sur les différen...,tableau,92,ASNIERES SUR SEINE,2023/07/13
3,Ingénieur Data H/F,"Vos missions\nAu sein de projets Agile Scrum, ...",66 - PERPIGNAN,Actualisé le 19 juillet 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,5 ans,NaN,"concevoir et gérer un projet, concevoir un log...",Cadre,"Autres intermédiaires du commerce en denrées, ...",SOCIETE DE DISTRIBUTION DE PRODUITS MARA,100 à 199 salariés,"Azura, est un groupe familial, en forte croiss...","tableau, sas",66,PERPIGNAN,2023/07/19
4,Data Engineer pour la Qualité Totale H/F (H/F),Vous êtes Rattaché(e) au Responsable de l'équi...,92 - ST CLOUD,Actualisé le 19 juillet 2023,35H Travail en journée,38000.0 - 70000.0,Contrat à durée indéterminée,5 ans,NaN,"application web, déterminer des mesures correc...",Cadre,Activités des sièges sociaux,DASSAULT AVIATION,Non employeur (pas de salarié au cours de l'an...,Dassault Aviation,NaN,92,ST CLOUD,2023/07/19


In [86]:
def concat_data(df):
    
    # importer le df qui contient toutes les données déjà récoltées
    df_history = pd.read_csv("../data/silver_data.csv")
    
    # concaténer les données qui viennent d'être récoltées à df_history
    global_df = pd.concat([df_history, df], axis=0)
    sort_df = global_df.sort_values('date', ascending=True)
    
    return sort_df

In [87]:
global_df = concat_data(df)

In [88]:
global_df.head()

,jobs,description,loc,datePosted,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description,tools,ID_dep,ville,date
0,Data Engineer sénior (F/H) CDI (H/F),"En tant que Data Engineer chez Quantmetry, vou...",75 - PARIS 08,Actualisé le 12 juillet 2023,38H Travail en journée,40000.0 - 50000.0,Contrat à durée indéterminée,5 ans,NaN,"concevoir et gérer un projet, concevoir un log...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
1,Data Consultant Stratégie Sénior (H/F),Nous recrutons des personnes avec une appétenc...,75 - PARIS 08,Actualisé le 12 juillet 2023,35H Travail en journée,3100.0 - 5000.0,Contrat à durée indéterminée,5 ans,NaN,"analyser les résultats d'un projet, décliner l...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
2,Chef de projets Performance Durable/Energie/Da...,Intégré(e) au sein de la Direction Performance...,92 - ASNIERES SUR SEINE,Actualisé le 13 juillet 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,3 ans,NaN,contrôler et faire appliquer le respect de dis...,Employé qualifié,Activités des sièges sociaux,NEXITY,250 à 499 salariés,Nexity est aujourd hui leader sur les différen...,tableau,92,ASNIERES SUR SEINE,2023/07/13
3,Ingénieur Data H/F,"Vos missions\nAu sein de projets Agile Scrum, ...",66 - PERPIGNAN,Actualisé le 19 juillet 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,5 ans,NaN,"concevoir et gérer un projet, concevoir un log...",Cadre,"Autres intermédiaires du commerce en denrées, ...",SOCIETE DE DISTRIBUTION DE PRODUITS MARA,100 à 199 salariés,"Azura, est un groupe familial, en forte croiss...","tableau, sas",66,PERPIGNAN,2023/07/19
4,Data Engineer pour la Qualité Totale H/F (H/F),Vous êtes Rattaché(e) au Responsable de l'équi...,92 - ST CLOUD,Actualisé le 19 juillet 2023,35H Travail en journée,38000.0 - 70000.0,Contrat à durée indéterminée,5 ans,NaN,"application web, déterminer des mesures correc...",Cadre,Activités des sièges sociaux,DASSAULT AVIATION,Non employeur (pas de salarié au cours de l'an...,Dassault Aviation,NaN,92,ST CLOUD,2023/07/19


In [89]:
global_df.shape

(3301, 19)

In [90]:
global_df.isnull().sum()

jobs                      0
description               0
loc                    2302
datePosted             2302
workhours              2697
salary                 2814
contract_type             0
experience                0
diploma                3185
skills                 3021
qualifications         1756
industry               1668
company                1556
size                   1130
company_description    1484
tools                  1627
ID_dep                    0
ville                   363
date                      0
dtype: int64

In [91]:
loc = global_df.loc[global_df.duplicated()]
loc.shape

(2, 19)

In [92]:
global_df = global_df.drop_duplicates()
global_df.duplicated().sum()

0

In [93]:
global_df["date"].min()

'2023/07/12'

In [94]:
global_df["date"].max()

'2023/08/31'

### Export silver df

In [99]:
def export_silver_df(df):
    csv_file_path = f"../data/silver/silver_data.csv"
    df.to_csv(csv_file_path, index=False)

In [100]:
export_silver_df(global_df)

In [102]:
df_final = pd.read_csv("../data/silver/silver_data.csv")
df_final.head()

,jobs,description,loc,datePosted,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description,tools,ID_dep,ville,date
0,Data Engineer sénior (F/H) CDI (H/F),"En tant que Data Engineer chez Quantmetry, vou...",75 - PARIS 08,Actualisé le 12 juillet 2023,38H Travail en journée,40000.0 - 50000.0,Contrat à durée indéterminée,5 ans,NaN,"concevoir et gérer un projet, concevoir un log...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
1,Data Consultant Stratégie Sénior (H/F),Nous recrutons des personnes avec une appétenc...,75 - PARIS 08,Actualisé le 12 juillet 2023,35H Travail en journée,3100.0 - 5000.0,Contrat à durée indéterminée,5 ans,NaN,"analyser les résultats d'un projet, décliner l...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
2,Chef de projets Performance Durable/Energie/Da...,Intégré(e) au sein de la Direction Performance...,92 - ASNIERES SUR SEINE,Actualisé le 13 juillet 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,3 ans,NaN,contrôler et faire appliquer le respect de dis...,Employé qualifié,Activités des sièges sociaux,NEXITY,250 à 499 salariés,Nexity est aujourd hui leader sur les différen...,tableau,92,ASNIERES SUR SEINE,2023/07/13
3,Ingénieur Data H/F,"Vos missions\nAu sein de projets Agile Scrum, ...",66 - PERPIGNAN,Actualisé le 19 juillet 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,5 ans,NaN,"concevoir et gérer un projet, concevoir un log...",Cadre,"Autres intermédiaires du commerce en denrées, ...",SOCIETE DE DISTRIBUTION DE PRODUITS MARA,100 à 199 salariés,"Azura, est un groupe familial, en forte croiss...","tableau, sas",66,PERPIGNAN,2023/07/19
4,Data Engineer pour la Qualité Totale H/F (H/F),Vous êtes Rattaché(e) au Responsable de l'équi...,92 - ST CLOUD,Actualisé le 19 juillet 2023,35H Travail en journée,38000.0 - 70000.0,Contrat à durée indéterminée,5 ans,NaN,"application web, déterminer des mesures correc...",Cadre,Activités des sièges sociaux,DASSAULT AVIATION,Non employeur (pas de salarié au cours de l'an...,Dassault Aviation,NaN,92,ST CLOUD,2023/07/19


In [103]:
df.shape

(688, 17)

In [27]:
def count_nan(df):
    nan_counts = df.isna().sum() # compte le nombre de NaN pour chaque colonne
    value_counts = df.count() # compte le nombre de valeurs non-NaN pour chaque colonne
    total_counts = len(df) # compte le nombre total de données dans le dataframe
    nan_percentages = (nan_counts / total_counts) * 100 # calcule le pourcentage de NaN pour chaque colonne
    result_df = pd.concat([value_counts, nan_counts, nan_percentages], axis=1) # combine les séries en un dataframe
    result_df.columns = ['True Values', 'NaN Count', 'NaN Percentage'] # renomme les colonnes du nouveau dataframe
    return result_df

In [28]:
NaN = count_nan(df_final)
NaN = NaN.sort_values(by = ['NaN Count'], ascending = False)
NaN

,True Values,NaN Count,NaN Percentage
diploma,88,2525,96.632223
skills,227,2386,91.312667
salary,388,2225,85.151167
workhours,523,2090,79.984692
loc,999,1614,61.768083
datePosted,999,1614,61.768083
qualifications,1154,1459,55.836204
industry,1206,1407,53.846154
tools,1306,1307,50.019135
company,1343,1270,48.603138
